In [1]:
from pyspark.sql import SparkSession

# create spark session
spark_session = SparkSession\
        .builder\
        .master("spark://hadoop-master:7077") \
        .appName("DE-1-4-SparkSession")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.executor.instances", 7) \
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max", 14) \
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 16:10:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import json
import random

# define ratio to start at 5%
ratio = 1/20

# load data into dataframe
entire_data = spark_session.read.json("hdfs://hadoop-master:9000/user/hadoop/input/corpus-webis-tldr-17.json")

# for loop to create seven subsets of the data
for i in range(7):
    # randomly sample the data
    sampled_data = entire_data.sample(False, ratio)
    # define the output path on the hdfs
    output_path = f"hdfs://hadoop-master:9000/user/hadoop/output/sampled_{5*(i+1)}_corpus-webis-tldr-17.json"
    # write the sampled data
    sampled_data.write.mode("overwrite").json(output_path)
    # increase ratio so the subsets are 5%, 10% ...
    ratio += 1/20


24/03/11 16:20:07 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/03/11 16:20:07 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce